<img src="image/multi-head.png" width="300" alt="multi-head.png">

<img src="image/scale_dot_product_attention.png" width="300" alt="attention.png">


In [ ]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_embed, Dropout=0.0): # 0% 概率dropout 
        super().__init__()
        assert d_embed % h == 0 # 确认H头数整除维度数
        self.d_k = d_embed // h # 每个头的输入维度
        self.h = h 
        self.WQ = nn.linear(d_embed, d_embed) # 注意过的是d_embed，后面才分割
        self.WK = nn.linear(d_embed, d_embed) # 注意过的是d_embed，后面才分割
        self.WV = nn.linear(d_embed, d_embed) # 注意过的是d_embed，后面才分割
        self.linear = nn.linear(d_embed, d_embed)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x_query, x_key, x_value, mask=None): # 训练的时候需要掩码，掩盖decoder未来的序列，防止decoder直接偷看未来的y造成训练作弊
        nbatch = x_query.size(0) # x_query的第一维度就是numbers of batch
        """now we project it to multihead q k v
        之前 x_query, x_key, x_value dimension: nbatch * seq_len * d_embed
        处理后 query, key, value dimensions: nbatch * h * seq_len * d_k

        """
        
        query = self.WQ(x_query).view(nbatch, -1, self.h, self.d_k).transpose(1,2) 
        key   = self.WK(x_key).view(nbatch, -1, self.h, self.d_k).transpose(1,2)
        value = self.WV(x_value).view(nbatch, -1, self.h, self.d_k).transpose(1,2)

        # .view的作用是调整tensor的形状
        # .transpose是改变tensor某两个维度的位置
        # Q 与 K 与 V 的形状（输入）： (B, H, S, D_k)  batch head seq_len dimention_per_head
        # Scores 的目标形状： (B, H, S, S) 
        # 所以要转置K的倒数第2 倒数第1 维度
        scores = torch.matmul(query, key.transpose(-2, -1))/math.sqrt(self.d_k)

        if mask is not None:
            scores = scores.masked_fill(mask, float('-inf'))

        # p_atten dimensions: nbatch * h * seq_len * seq_len
        p_atten = torch.nn.functional.softmax(scores, dim=-1)
        p_atten = self.dropout(p_atten)

        # x dimensions: nbatch * h * seq_len * d_k
        x = torch.matmul(p_atten, value)

        # x now has dimensions:nbtach * seq_len * d_embed
        x = x.transpose(1, 2).contiguous().view(nbatch, -1, self.d_embed)
        return self.linear(x) # final linear layer



<img src="image/transformer.png" width="300">

In [ ]:
class ResidualConnection(nn.Module):
  '''residual connection: x + dropout(sublayer(layernorm(x))) '''
  def __init__(self, dim, dropout):
      super().__init__()
      self.drop = nn.Dropout(dropout)
      self.norm = nn.LayerNorm(dim)

  def forward(self, x, sublayer): # sublayer之后在encoder   decoder里面会传进去
      return x + self.drop(sublayer(self.norm(x))) 

In [ ]:
class Encoder(nn.Module):
    '''Encoder = token embedding + positional embedding -> a stack of N EncoderBlock -> layer norm'''
    def __init__():
        super().__init__()
        self.d_embed = config.d_embed
        self.tok_embed = nn.Embedding(config.encoder_vocab_size, config.d_embed) # embedding的定义是MLP
        self.pos_embed = nn.Parameter(torch.zeros(1, config.max_seq_len, config.d_embed)) 
        # parameter定义是一组自己学习的参数，没有使用正弦余弦编码了这里
        self.encoder_blocks = nn.ModuleList([EncoderBlock(config) for _ in range(config.N_encoder)])
        # 创建很多个module实例
        self.dropout = nn.Dropout(config.dropout)
        self.norm = nn.LayerNorm(config.d_embed)


    def forward (self, input, mask=None):
        x = self.tok_embed(input)
        x_pos = self.pos_embed[:, :x.size(1), :] 
        # 为了获得一个形状为 (1, 实际序列长度, config.d_embed) 的张量 x_pos
        # 然后将它与输入张量 x 相加，从而为输入序列的每个词添加其位置信息
        
        x = self.dropout(x + x_pos) # 加上位置编码
        for layer in self.encoder_blocks:
            x = layer(x, mask)
        return self.norm(x)


class EncoderBlock(nn.Module):
    '''EncoderBlock: self-attention -> position-wise fully connected feed-forward layer'''
    def __init__(self, config):
        super(EncoderBlock, self).__init__()
        self.atten = MultiHeadedAttention(config.h, config.d_embed, config.dropout)
        self.feed_forward = nn.Sequential(
            nn.Linear(config.d_embed, config.d_ff),
            nn.ReLU(),
            nn.Dropout(config.dropout),
            nn.Linear(config.d_ff, config.d_embed)
        )
        self.residual1 = ResidualConnection(config.d_embed, config.dropout)
        self.residual2 = ResidualConnection(config.d_embed, config.dropout)

    def forward(self, x, mask=None):
        # self-attention
        x = self.residual1(x, lambda x: self.atten(x, x, x, mask=mask))
        # position-wise fully connected feed-forward layer
        return self.residual2(x, self.feed_forward)

<img src="image/transformer.png" width="300">

In [ ]:
class Decoder(nn.Module):
    '''Decoder = token embedding + positional embedding -> a stack of N DecoderBlock -> fully-connected layer'''
    def __init__(self, config):
        super().__init__()
        self.d_embed = config.d_embed
        self.tok_embed = nn.Embedding(config.decoder_vocab_size, config.d_embed)
        self.pos_embed = nn.Parameter(torch.zeros(1, config.max_seq_len, config.d_embed))
        self.dropout = nn.Dropout(config.dropout)
        self.decoder_blocks = nn.ModuleList([DecoderBlock(config) for _ in range(config.N_decoder)])
        self.norm = nn.LayerNorm(config.d_embed)
        self.linear = nn.Linear(config.d_embed, config.decoder_vocab_size)


    def future_mask(self, seq_len):
        '''mask out tokens at future positions'''
        mask = (torch.triu(torch.ones(seq_len, seq_len, requires_grad=False), diagonal=1)!=0).to(DEVICE)
        # torch.ones 生成单位阵
        # torch.triu 上三角矩阵
        return mask.view(1, 1, seq_len, seq_len) # 拓展形状（广播，前面两个是B H）

    def forward(self, memory, src_mask, trg, trg_pad_mask):
        seq_len = trg.size(1)
        trg_mask = torch.logical_or(trg_pad_mask, self.future_mask(seq_len)) # 未来的掩码
        x = self.tok_embed(trg) + self.pos_embed[:, :trg.size(1), :]
        x = self.dropout(x)
        for layer in self.decoder_blocks:
            x = layer(memory, src_mask, x, trg_mask)
        x = self.norm(x)
        logits = self.linear(x) # 先不过softmax
        return logits

In [ ]:
class DecoderBlock(nn.Module):
    ''' EncoderBlock: self-attention -> position-wise feed-forward (fully connected) layer'''
    def __init__(self, config):
        super().__init__()
        self.atten1 = MultiHeadedAttention(config.h, config.d_embed)
        self.atten2 = MultiHeadedAttention(config.h, config.d_embed)
        self.feed_forward = nn.Sequential(
            nn.Linear(config.d_embed, config.d_ff),
            nn.ReLU(),
            nn.Dropout(config.dropout),
            nn.Linear(config.d_ff, config.d_embed)
        )
        self.residuals = nn.ModuleList([ResidualConnection(config.d_embed, config.dropout)
                                       for i in range(3)])

    def forward(self, memory, src_mask, decoder_layer_input, trg_mask):
        x = memory # 传进来的 是 encoder 的输出，含有 k 和 v 信息的latent
        y = decoder_layer_input
        y = self.residuals[0](y, lambda y: self.atten1(y, y, y, mask=trg_mask))
        # 上面是triangle掩码，就是因果掩码
        # lambda表达式的作用式是一个local function，等到调用这个lambda的时候再进行lambda表达式的计算
        # 也即是那个residual里面的sublayer
        # 这里q k v 传的都是y 所以是self-attention
        y = self.residuals[1](y, lambda y: self.atten2(y, x, x, mask=src_mask))
        # q 传的是decoder的
        # src_mask作用是去掉padding
        return self.residuals[2](y, self.feed_forward)


class Transformer(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, src_mask, trg, trg_pad_mask):
        return self.decoder(self.encoder(src, src_mask), src_mask, trg, trg_pad_mask)

    

## 掩码工作原理示例

这是一个针对 Transformer 训练中三种核心掩码（Mask）的详细解析。

### 场景设定

假设我们正在训练一个翻译模型：**[英文] $\rightarrow$ [中文]**，且批次最大长度为 5。

| Sequence | 原始长度 | 填充后序列 (IDs) |
| :--- | :--- | :--- |
| **SRC** (英文) | 3 | $s_1, s_2, s_3, \text{PAD}, \text{PAD}$ |
| **TRG** (中文) | 4 | $t_1, t_2, t_3, t_4, \text{PAD}$ |

我们关注 TRG 序列的**第一个位置 $t_1$** 在解码器中计算注意力时的状态。

---

### 1. `src_mask` (源序列填充掩码)

* **作用位置：** 解码器的**交叉注意力**层（Query $\in$ TRG, Key/Value $\in$ SRC）。
* **创建依据：** SRC 序列中的 `<pad>` 位置。
* **掩码形状 (1x5)：** `[False, False, False, True, True]`
* **原理：** 当 $t_1$ 查询 SRC 信息时，`src_mask` 强制它对 $s_4, s_5$ (PAD) 的注意力权重为 **0**。

### 2. `trg_pad_mask` (目标序列填充掩码)

* **作用位置：** 解码器的**自注意力**层。
* **创建依据：** TRG 序列中的 `<pad>` 位置。
* **掩码形状 (1x5)：** `[False, False, False, False, True]`
* **原理：** 确保 TRG 序列中的任何词（包括 $t_1$）都不会对 TRG 序列末尾的 $\text{PAD}$ 产生注意力。

### 3. `future_mask` (前瞻掩码)

* **作用位置：** 解码器的**自注意力**层。
* **创建依据：** 序列的几何结构（上三角矩阵）。与 PAD 无关。
* **原理：** 强制每个位置只能关注它自己和它之前的词，维护自回归特性。
* **$t_1$ 所在行（Query $\in t_1$）的视图：**
    
| Q $\downarrow$ / K $\rightarrow$ | $t_1$ | $t_2$ | $t_3$ | $t_4$ | $\text{PAD}$ |
| :---: | :---: | :---: | :---: | :---: | :---: |
| **$t_1$** | **F** | **T** | **T** | **T** | **T** |

---

### 4. 联合掩码 (`trg_mask`)

**计算方式：** $\text{trg\_mask} = \text{trg\_pad\_mask} \lor \text{future\_mask}$ (逻辑或)

**$t_1$ 的最终状态：**

1.  `future_mask` 已经将 $t_2$ 到 $\text{PAD}$ 的所有位置标记为 `True`（屏蔽）。
2.  `trg_pad_mask` 仅将 $\text{PAD}$ 标记为 `True`。

由于逻辑或运算，最终 $t_1$ 行的注意力矩阵中，**只有 $t_1$ 自身位置**被标记为 `False`（允许关注），其他所有位置都被标记为 `True`（屏蔽）。这确保了 $t_1$ 的输出预测**只基于其自身信息**（以及 $\text{<sos>}$ 标记），符合严格的自回归约束。